In [1]:
import os
import random
import re
import numpy as np
import shutil
import pandas as pd

In [2]:
rng = np.random.default_rng(seed=152) # old number=42, 

In [3]:
def add_text_to_file(file_path, line_number, text_to_add):
    # Read the contents of the file
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Insert the new text at the specified line number
    lines.insert(line_number - 1, text_to_add + '\n')

    # Write the modified contents back to the file
    with open(file_path, 'w') as file:
        file.writelines(lines)

In [4]:
def change_lines(file_path, lines_to_change, changed_lines):
    # Read the contents of the file
    with open(file_path, 'r') as file:
        lines = file.readlines()

    i = 0
    for idx, line in enumerate(lines):
        if re.search(lines_to_change[i], line):
            lines[idx] = changed_lines[i]
            i+=1
            if i == len(changed_lines):
                break
    if len(changed_lines) != i:
        print(len(changed_lines) - i, "lines not changed")
     
    # Write the modified contents back to the file
    with open(file_path, 'w') as file:
        file.writelines(lines)

In [5]:
# Get the current directory
current_dir = os.getcwd()

file_path_initial = "/app/nse/resources/config-plume-ex.txt"
file_path_output = "/app/nse/outputs/"


In [6]:
# Number of passive tracers
n_files = 100
N = 128

In [7]:
# features

# Size of model [m]
Height_model_z = 1e3
Length_model_x = 2e3
Width_model_y  = 2e3

x = Length_model_x * 0.25

y_min = Width_model_y  * 0.3
y_max = Width_model_y  * 0.7

z_min = Height_model_z * 0.05
z_mode = Height_model_z * 0.3
z_max = Height_model_z * 0.7

# Velocity x [m/s]
u_min = 0.5
u_max = 15
# power of tracers
p_min = 0.8
p_max = 1.7
# roughness [m]
z0_min = 0.001
z0_max = 0.07
# field temperature [K]
T0_min = 230
T0_max = 300
# value [K*m/s]
value_min = -0.5
value_max = -0.1
# temperature gradient [K/m]
gradz_min = 0.001
gradz_max = 0.1


In [8]:
y_list = []
z_list = []
u_list = []
p_list = []
z0_list = []
T0_list = []
value_list = []
gradz_list = []

In [9]:
# The initial line of the writing
initial_line = 332
lines_to_change = [
    'U = 7.5; V = 0.0;',
    'value = -0.35;',
    'z0_m = 0.1;',
    'surface_value = 241.0;',
    'grad_z = 0.03;',
    "num = 1;"
]

In [10]:

for idx in range(n_files):
	
	file_path = shutil.copyfile(file_path_initial, file_path_output + "config-plume-ex_" + str(idx) + ".txt")
	
	u = rng.uniform(u_min, u_max)
	value = rng.uniform(value_min, value_max)
	z0_m = rng.uniform(z0_min, z0_max)
	T0 = rng.uniform(T0_min, T0_max)
	gradz = rng.uniform(gradz_min, gradz_max)

	lines_changed = [
		f'	U = {u}; V = 0.0;			# [m/s] \n',
		f'		value = {value};	# sensible heat flux [K*m/s]\n',
		f'	z0_m = {z0_m};		# aerodynamic roughness [m]\n',
		f'		surface_value = {T0};	# initial boundary layer temperature [K]\n',
		f'		grad_z = {gradz};		# temperature gradient above boundary layer [K/m]\n',
		f'	num = {N};	# number of tracers, skipped if not defined'
	]
	for i in range(N, 0, -1):
		p = rng.uniform(p_min, p_max)
		y = rng.uniform(y_min, y_max)
		z = rng.triangular(z_min, z_mode, z_max)
		
		y_list.append(y)
		z_list.append(z)
		u_list.append(u)
		value_list.append(value)
		z0_list.append(z0_m)
		T0_list.append(T0)
		gradz_list.append(gradz)
		p_list.append(p)

		# Define the text to add
		text_to_add = f"""
	tracer_{i} {{ 
		diffusivity = phys.xi;

		surface {{ 
			flux = 0.0;
		}}

		# --- point emission source [optional]
		point_emission {{
			type = "gaussian";		# "gaussian" || "box"

			# --- source intensity [ppb * m^3 / s]
			value = {p} * (2.0 / 3.14) * (1.0 / 2.46) * 1000.0 * 1000.0 * 100.0;

			# --- active in [begin, end], [time.begin, time.end] if not defined
			begin = 0.5 * 3600.0;	# [s]

			xpos = domain.x + 0.25 * domain.length;		# [m]
			ypos = domain.y + {y};		# [m]
			zpos = {z};		# [m]
			
			sx = 50.0;		# [m]
			sy = 50.0;		# [m]
			sz = 25.0;		# [m]

			# --- OR set 'box'
			# type = "box";
			
			# xmin = 450.0; xmax = 550.0;	# [m]
			# ymin = 450.0; ymax = 550.0;	# [m]
			# zmin = 50.0; zmax = 250.0;	# [m]
		}}

		# --- boundary conditions [optional]
		boundary_conditions
		{{
			# --- default: -xy periodic & neumann (= 0) at top and bottom
			west {{ type = "inout"; rhs = 0.0; }}
			east {{ type = "inout"; rhs = 0.0; }}
			south {{ type = "inout"; rhs = 0.0; }}
			north {{ type = "inout"; rhs = 0.0; }}
		}}

		# --- sedimentation & deposition [optional, default = false]
		# 	--- require density & diameter setup
		# is_sedimentation = false;
		# is_dry_deposition = false;
		# is_wet_deposition = false;
		# density = 10.0 * 1000.0;	# [kg / m^3]
		# diameter = 10.0 * 0.000001;	# [m]
		
		# --- min limit [optional, default = none]
		# min_value = 0.0;
	}}
		"""
		
		# Add the text to initial line
		add_text_to_file(file_path, initial_line, text_to_add)
	change_lines(file_path, lines_to_change, lines_changed)
print(f"Text added successfully to {file_path} at line 346.")

Text added successfully to /app/nse/outputs/config-plume-ex_99.txt at line 346.


In [11]:
features_pd = pd.DataFrame({"y": y_list,
                            "z": z_list,
                            "u": u_list,
                            "power":p_list,
                            "roughness": z0_list,
                            "T": T0_list,
                            "sensible_heat_flux":value_list,
                            "T_grad":gradz_list})
features_pd.to_csv("features.csv", index=False)